<a href="https://colab.research.google.com/github/abishekravi/ML-projects/blob/master/lungs3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [2]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '







17227776/17225924 [==============================] - 0s 0us/step


In [0]:

model=Model(inputs=base_model.input,outputs=preds)

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [6]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abishek003","key":"f00110dc945d3e1f7438bd66503927a2"}'}

In [7]:

!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:22<00:00, 109MB/s]
100% 2.29G/2.29G [00:22<00:00, 111MB/s]


In [0]:
import zipfile
with zipfile.ZipFile('/content/chest-xray-pneumonia.zip','r') as zip_ref:
  zip_ref.extractall('/content/chest_xray')

In [0]:
import csv


train_dir = '/content/chest_xray/chest_xray/train' # Path to the train directory
class_dirs = [i for i in os.listdir(path=train_dir) if os.path.isdir(os.path.join(train_dir, i))]
with open('/content/chest_xray/chest_xray/train/label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    label = 0
    for current_class in class_dirs:
        for image in os.listdir(os.path.join(train_dir, current_class)):
            writer.writerow({'File Name': str(image), 'Label':label})
        label += 1
    train_csv.close()

In [0]:

import pandas as pd
# import formatter as fm
df = pd.read_csv('/content/chest_xray/chest_xray/train/label.csv')
# preprocess_input, test_data = fm(df)
preprocess_input, test_data = df

In [11]:
train_datagen=ImageDataGenerator(rescale = 1./255,
                                 shear_range = 0.2,
                                 zoom_range = 0.2,
                                 rotation_range=90,
                                 horizontal_flip=False,
                                 vertical_flip=True) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/chest_xray/chest_xray/train', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator(rescale = 1./255)  #Image normalization.


validation_generator = test_datagen.flow_from_directory('/content/chest_xray/chest_xray/val',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_set = test_datagen.flow_from_directory('/content/chest_xray/chest_xray/test',
                                            target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                         steps_per_epoch = step_size_train,
                         epochs = 1,
                         validation_data = validation_generator,
                         validation_steps = 624)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
163/163 [==============================] - 2233s 14s/step - loss: 0.2206 - acc: 0.9212 - val_loss: 7.3739 - val_acc: 0.5000


In [0]:

model.save('chest-xray-pneumonia.h5')    
#!zip -r model.zip 'chest-xray-pneumonia.h5'

In [0]:
from keras.models import load_model
new_model = load_model("/content/chest-xray-pneumonia.h5")

def get_rez(pic):
  img = image.load_img(pic, target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  p_good,p_ill = np.around(new_model.predict(x), decimals=2)[0]
  return{'p_good':p_good,'p_ill':p_ill}

In [105]:
ill_path = "/content/chest_xray/chest_xray/train/PNEUMONIA/" 
good_path = "/content/chest_xray/chest_xray/train/NORMAL/" 

ill_pic = ill_path + os.listdir(ill_path)[52]
good_pic = good_path + os.listdir(good_path)[41]

print(get_rez(ill_pic))
print(get_rez(good_pic))

{'p_good': 0.0, 'p_ill': 1.0}
{'p_good': 0.62, 'p_ill': 0.38}


In [0]:
model_json = model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)

In [0]:
files.download("model.json")

In [0]:
model.save_weights("model.h5")

In [0]:
files.download("model.h5")

In [50]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory
model.save('model.json')    
model_file = drive.CreateFile({'title' : 'model.json'})
model_file.SetContentFile('model.json')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1h15Fvk890Ar-q3nRrPIE9fIrhQdegBWZ'})